# Análasis de dependencia de Quimicos - RecuperacionAzucar

Se desea validar la dependencia de los quimicos para la producción de unos procesos, y el impacto en las etapas del proceso.

El objetivo sera demostrar con analisis estadistico el nivel de dependencia de las variables, los valores que hacen que incremente o disminuya.

### Cargamos los datos

In [2]:
import pandas as pd
import numpy as np
import sklearn.model_selection as skmodel
import warnings
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
import sklearn.metrics as skmetrics
from sklearn.model_selection import KFold, cross_validate
import sklearn.model_selection as skmodel
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.filterwarnings("ignore")
from statsmodels.formula.api import ols
import statsmodels.api as sm

import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

import math
import cufflinks as cf
cf.set_config_file(sharing='public', theme='ggplot', offline=True)
cf.getThemes()

['ggplot', 'pearl', 'solar', 'space', 'white', 'polar', 'henanigans']

In [3]:
dataset = pd.read_excel("./DataSet_ValidaQuimicos.xlsx", header=3, sheet_name='Q_RecuperacionAzucar')
dataset= dataset.drop('Y_TamañoCristalSlurry_19_+', axis=1)
dataset=dataset.dropna()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185 entries, 2 to 186
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Semana                                    185 non-null    int64         
 1   Fecha                                     185 non-null    datetime64[ns]
 2   dia                                       185 non-null    int64         
 3   Tensoactivo (kg/t Azúcar)                 185 non-null    float64       
 4   Alcohol Isopropílico (l/t Azúcar)         185 non-null    float64       
 5   Carbón Activado (kg/t Azúcar Refino)      185 non-null    float64       
 6   Tierra de Diatomeas (kg/t Azúcar Refino)  185 non-null    float64       
 7   Sacarasa (kg/t de HTM)                    185 non-null    float64       
 8   Cal [kg/T Az. Refino]                     185 non-null    float64       
 9   Floculante [kg/T Az. Refino]    

#### Agregacion por semana

In [4]:
df_semana = dataset.astype({"Semana": int})
df_semana = df_semana.groupby(by=["Semana"], dropna=False).mean()
df_semana.reset_index(inplace= True)
df_semana

,Semana,dia,Tensoactivo (kg/t Azúcar),Alcohol Isopropílico (l/t Azúcar),Carbón Activado (kg/t Azúcar Refino),Tierra de Diatomeas (kg/t Azúcar Refino),Sacarasa (kg/t de HTM),Cal [kg/T Az. Refino],Floculante [kg/T Az. Refino],Decolorante [L/T Az. Refino],...,I_DECOLORANTE,Y_BrixMasaSalidaCVC_18_+,Y_AzucaresTotaleHTM_22_+,YCal_RemocionTurbidez_23_+,YCal_pH_23_+,YF_RemocionTurbidez_24_+,YD_RemocionColor_25_+,YC_RemocionColor_20_+,YC_pH_20_+,YT_RemocionTurbidez_21_+
0,1,3.0,0.000000,250.000000,125.000000,567.500000,66.666667,625.000000,104.000000,0.000000,...,0.000000,0.000000,61.864167,565.005311,5.482222,565.005311,247.510689,247.510689,5.482222,565.005311
1,2,8.0,28.857143,307.142857,297.857143,937.185714,102.857143,642.857143,80.142857,0.000000,...,0.000000,0.000000,56.263890,489.704357,5.626429,489.704357,140.789607,140.789607,5.626429,489.704357
2,3,15.0,4.285714,322.857143,229.285714,1277.685714,97.142857,535.714286,108.857143,0.000000,...,0.000000,27.424286,52.752607,374.853302,5.257500,374.853302,55.786090,55.786090,5.257500,374.853302
3,4,22.0,38.285714,270.000000,252.857143,1365.142857,134.285714,589.285714,99.571429,0.000000,...,0.000000,82.387143,55.691476,364.036733,5.538810,364.036733,56.438095,56.438095,5.538810,364.036733
4,5,29.0,70.000000,182.857143,233.571429,1378.214286,174.285714,596.428571,130.571429,0.000000,...,0.000000,40.708333,57.154821,333.666667,5.633571,333.666667,66.583333,66.583333,5.633571,333.666667
5,6,36.0,30.000000,202.857143,317.142857,1290.657143,177.142857,578.571429,113.285714,0.000000,...,0.000000,82.116857,56.141190,271.242857,5.580595,271.242857,48.747619,48.747619,5.580595,271.242857
6,7,43.0,28.571429,192.572857,357.857143,1267.957143,162.857143,589.285714,113.857143,0.000000,...,0.000000,96.289048,56.243090,185.046488,5.571976,185.046488,51.415833,51.415833,5.571976,185.046488
7,8,50.0,55.714286,228.807143,144.285714,1303.628571,185.714286,510.714286,102.285714,0.000000,...,0.000000,96.112381,56.881714,185.345238,5.650000,185.345238,13.655869,13.655869,5.650000,185.345238
8,9,57.0,18.571429,236.515714,117.142857,1125.271429,202.857143,603.571429,112.142857,0.000000,...,0.000000,95.894286,56.754452,164.923810,5.534405,164.923810,22.219048,22.219048,5.534405,164.923810
9,10,64.0,2.857143,222.151429,145.714286,1076.628571,197.142857,557.142857,95.428571,0.000000,...,0.000000,95.681905,56.866869,163.000000,5.591905,163.000000,28.272917,28.272917,5.591905,163.000000


## Definimos las Variables (Y's) a Analizar

In [5]:
def get_ys(df):
    ys=[]
    for i in df.iloc[:,25:].columns:
        if not (i.__contains__('X')):
            ys.append(i)
    return ys

In [6]:
ys=get_ys(df_semana)
ys

['Y_BrixMasaSalidaCVC_18_+',
 'Y_AzucaresTotaleHTM_22_+',
 'YCal_RemocionTurbidez_23_+',
 'YCal_pH_23_+',
 'YF_RemocionTurbidez_24_+',
 'YD_RemocionColor_25_+',
 'YC_RemocionColor_20_+',
 'YC_pH_20_+',
 'YT_RemocionTurbidez_21_+']

## Lista pares Variable vs Químico

Se realizo una función para poder visualizar los pares Variable vs Químico previo a su análisis.

In [7]:
def list_quimicos(df):
    ys = get_ys(df)
    lista_quimicos = []
    lista_quimicos_completa = []
    for y in ys:
        sep = y.split(sep='_')
        name_y = '_'.join(sep[1:2])
        posx= int(sep[-2])-1
        complete_x_name = df.iloc[:,posx:posx+1].columns[0]
        name_x = ''.join((df.iloc[:,posx:posx+1].columns[0]).split(sep='_')[1:])
        lista_quimicos.append(name_y + ' ~ ' + name_x)
        lista_quimicos_completa.append(y + ' ~ ' + complete_x_name)

    return lista_quimicos, lista_quimicos_completa

In [8]:
list_quimicos(df_semana)

(['BrixMasaSalidaCVC ~ TENSOACTIVO',
  'AzucaresTotaleHTM ~ SACARASA',
  'RemocionTurbidez ~ CAL',
  'pH ~ CAL',
  'RemocionTurbidez ~ FLOCULANTE',
  'RemocionColor ~ DECOLORANTE',
  'RemocionColor ~ CARBONACTIVADO',
  'pH ~ CARBONACTIVADO',
  'RemocionTurbidez ~ TIERRADIATOMEAS'],
 ['Y_BrixMasaSalidaCVC_18_+ ~ I_TENSOACTIVO',
  'Y_AzucaresTotaleHTM_22_+ ~ I_SACARASA',
  'YCal_RemocionTurbidez_23_+ ~ I_CAL',
  'YCal_pH_23_+ ~ I_CAL',
  'YF_RemocionTurbidez_24_+ ~ I_FLOCULANTE',
  'YD_RemocionColor_25_+ ~ I_DECOLORANTE',
  'YC_RemocionColor_20_+ ~ I_CARBON_ACTIVADO',
  'YC_pH_20_+ ~ I_CARBON_ACTIVADO',
  'YT_RemocionTurbidez_21_+ ~ I_TIERRA_DIATOMEAS'])

## Analisis general grafico de la dependencia de variables

**TABLA PENDIENTE DE ACTUALIZAR**

In [10]:
datos = {
    'Indice' : ['I_TENSOACTIVO','I_ALCOHOL_ISOPROPILICO','I_SACARASA','I_CAL','I_FLOCULANTE','I_DECOLORANTE','I_CARBON_ACTIVADO','I_TIERRA_DIATOMEAS'],
    'Nombre' : ['Tensoactivo [kg/T Azucar]','Alcohol [L/T Az Total]', 'Sacarasa para HTM [kg/T HTM]', 'Cal [kg/T Az. Refino]','Floculante [kg/T Az. Refino]','Decolorante [L/T Az. Refino]','Carbón Activado[kg/T Azúcar Refino]','Tierra Diatomea [kg/T Azúcar Refino]'],
    'Indice_Meta': [0.0277,0.1,0.35,0.54,0.01,0.08,0.3,1.1]
}

df_indices = pd.DataFrame(datos)
df_indices

,Indice,Nombre,Indice_Meta
0,I_TENSOACTIVO,Tensoactivo [kg/T Azucar],0.0277
1,I_ALCOHOL_ISOPROPILICO,Alcohol [L/T Az Total],0.1000
2,I_SACARASA,Sacarasa para HTM [kg/T HTM],0.3500
3,I_CAL,Cal [kg/T Az. Refino],0.5400
4,I_FLOCULANTE,Floculante [kg/T Az. Refino],0.0100
5,I_DECOLORANTE,Decolorante [L/T Az. Refino],0.0800
6,I_CARBON_ACTIVADO,Carbón Activado[kg/T Azúcar Refino],0.3000
7,I_TIERRA_DIATOMEAS,Tierra Diatomea [kg/T Azúcar Refino],1.1000


In [13]:
import plotly.figure_factory as ff
def graphs_multi(df,y):
    sep=y.split(sep='_')
    name_y='_'.join(sep[:2])
    posx= int(sep[-2])-1
    name_x=df.iloc[:,posx:posx+1].columns[0]
    data_clean=df.drop(df[df[name_x]==0].index)
    q3=data_clean[name_x].quantile(0.75)
    q1=data_clean[name_x].quantile(0.25)
    iqr=q3-q1
    upper=q3+1.75*iqr
    lower=q1-1.75*iqr
    outliers_indx=data_clean[(data_clean[name_x]>upper) | (data_clean[name_x]<lower) | (data_clean['Caña Recibida']<23000)].index
    data_clean=data_clean.drop(outliers_indx)
    
    x1 = data_clean[name_x]
    y1 = data_clean[y]
    ignore=['X_FC_BrixMeladuraEvaporada', 'X_D_BrixMeladuraEvaporada', 'I_AZUFRE_CAÑA', 'I_ACIDO_SULFAMICO']
    stat, p = scipy.stats.pearsonr(x1, y1)
    print("Coeficiente de Correllacion de Pearson: ",stat)
    print("p-value: ",p)
    
    
    fig = make_subplots(
                rows=3, cols=2,
                vertical_spacing=0.08,
                specs=[[{'type': 'scatter'}, {'type':'scatter'}],
                       [{'type': 'histogram'}, {'type':'histogram'}],
                       [{'type': 'box'}, {'type':'box'}]]
            )
    
    fig.add_trace(go.Scatter(x=x1, y=y1, mode='markers', showlegend=False, marker_color='orange'), row=1,col=1)
    if not (ignore.__contains__(name_x)):
        meta_indice = df_indices.loc[df_indices['Indice']== name_x]['Indice_Meta'].values[0]
        fig.add_vline(x=meta_indice,line_width=3, line_dash="dash", line_color="#6699ff", annotation_text='Meta= '+str(meta_indice), annotation_position='top')
    fig.add_trace(go.Histogram2dContour(x=x1, y=y1, colorscale='Blues', reversescale=False, showscale=False), row=1, col=1)    
    fig.update_xaxes(title_text=name_x, dtick=(max(x1)-min(x1))/10,row=1, col=1)
    fig.update_yaxes(title_text=name_y, row=1, col=1)
    fig.add_trace(go.Scatter(x=data_clean['Semana'], y=x1, mode='markers', marker_symbol='x', name=name_x, marker_color='#6699ff'), row=1,col=2)
    fig.add_trace(go.Scatter(x=data_clean['Semana'], y=y1, mode='markers', name=name_y, marker_color='#8fbc8f'), row=1,col=2)
    fig.update_xaxes(title_text='Semana', dtick=1,row=1, col=2)
    fig.update_yaxes(title_text='Valor', row=1, col=2)
    fig.add_trace(go.Histogram(x=x1, name=name_x, nbinsx=20,showlegend=False, marker_color='#6699ff',marker_line_width=1, marker_line_color='white'), row=2, col=1)
    fig.update_xaxes(title_text=name_x,row=2, col=1)
    fig.add_trace(go.Histogram(x=y1, name=name_y,nbinsx=20, showlegend=False, marker_color='#8fbc8f',marker_line_width=1, marker_line_color='white'), row=2, col=2)
    fig.update_xaxes(title_text=name_y, row=2, col=2)
    fig.add_trace(go.Box(y=x1, showlegend=False, marker_color='#6699ff'), row=3, col=1)
    fig.update_xaxes(title_text=name_x, row=3, col=1)
    fig.add_trace(go.Box(y=y1, showlegend=False, marker_color='#8fbc8f'), row=3, col=2)
    fig.update_xaxes(title_text=name_y, row=3, col=2)
    fig.update_layout(height=1300,hovermode='x', legend=dict(orientation="h",
                                                            yanchor="bottom",
                                                            y=1,
                                                            xanchor="right",
                                                            x=1))
    fig.show()

In [14]:
for i in ys:
    graphs_multi(df_semana, i)

Coeficiente de Correllacion de Pearson:  -0.28891218337325736
p-value:  0.17092880076434377


Coeficiente de Correllacion de Pearson:  0.22223425219302848
p-value:  0.39128087629987146


Coeficiente de Correllacion de Pearson:  0.022324180758419743
p-value:  0.915646466292832


Coeficiente de Correllacion de Pearson:  0.044931842352123406
p-value:  0.8311204087863465


Coeficiente de Correllacion de Pearson:  -0.13984184214595083
p-value:  0.5049636146984443


Coeficiente de Correllacion de Pearson:  0.12144463732682761
p-value:  0.7556156425893971


Coeficiente de Correllacion de Pearson:  0.531532471562951
p-value:  0.00624967755076295


Coeficiente de Correllacion de Pearson:  -0.30922910769426026
p-value:  0.13254669799479915


Coeficiente de Correllacion de Pearson:  0.36709322383947945
p-value:  0.07105642441876225


## Con un scatter plot, graficamos la correlación entre cada Variable a analizar y su respectivo Químico

Se realizaron ciertos filtros y limpieza en los datos.

#### **Outliers:**

1-. **Ceros:** El primer paso es limpiar los datos eliminando todas las filas del par Variable vs Químico, donde la variable contenga un valor igual a 0, ya que representa que no se tomo medida ese día.

2-. **Outliers:** Se identificaron los outliers existentes en cada Variable vs Químico, con un criterio por rango intercuartilico. Se considera un outlier aquel valor que cumpla con una de las sigueintes dos condiciones: 
* El valor es menor que el primer cuartil menos 1.75 veces el rango intercuartilico.
* El valor es mayor que el tercer cuartil más 1.75 veces el rango intercuartilico.

3-. **Caña Recibida:** Se identificaron los casos donde la Caña Recibida no estuvo dentro del rango aceptable o promedio por día. El valor para escoger estos casos fue todo aquel arriba de 23,000. Esto se realizo dado que si no se obtuvo al menos un valor de 23,000 en caña recibida, es muy probable que haya ocurrido imprevistos en producción, como apagones u algun otro inconveniente, convirtiendo esos casos en otro tipo de outliers.

**OBSERVACION:** En cada gráfica se podrá observar el valor de la correlación encontrada.

In [14]:
def graph_corr(df,y):
    sep=y.split(sep='_')
    name_y='_'.join(sep[:2])
    posx= int(sep[-2])-1
    name_x=df.iloc[:,posx:posx+1].columns[0]
    data_clean=df.drop(df[df[name_x]==0].index)
    q3=data_clean[name_x].quantile(0.75)
    q1=data_clean[name_x].quantile(0.25)
    iqr=q3-q1
    upper=q3+1.75*iqr
    lower=q1-1.75*iqr
    outliers_indx=data_clean[(data_clean[name_x]>upper) | (data_clean[name_x]<lower) | (data_clean['Caña Recibida']<23000)].index
    data_clean=data_clean.drop(outliers_indx)
    fig = px.scatter(data_clean, x=name_x, y=i,labels={i:name_y})
    fig.add_trace(go.Indicator(mode="number",
                                value=np.corrcoef(data_clean[name_x],data_clean[i])[0,1],
                                title={'text':'Correlación'},
                                domain = {'y': [0.85, 0.9], 'x': [0.92, 1]}))

    fig.show()

In [15]:
import plotly.express as px
df = px.data.iris()
for i in ys:
    graph_corr(df_semana,i)

## Graficamos la serie de tiempo correspondiente a cada par Variable a analizar.

Se decidio realizar este tipo de gráfica dado que es importante comprender como es el desarrollo y comportamiento de cada variable a lo largo de la Zafra.

**OBSERVACION**: En este caso especifico de las series de tiempo, no se elimino ningun outlier, ya que lo que importa ver en este tipo de gráficas es el desarrollo diario de las Variables y poder determinar si existen patrones respecto al tiempo.

In [16]:
for i in ys:
    sep=i.split(sep='_')
    name_y='_'.join(sep[:2])
    y_plot=df_semana[i]
    y_plot=y_plot.reset_index().rename(columns={'index':'SEMANA'})
    fig = px.line(y_plot,x='SEMANA', y=i)
    fig.update_layout(
                title= 'Time Series of ' + name_y,
                yaxis_title=name_y,
                hovermode='x'
                
    )
    fig.show()

## Graficando la distribución de puntos para cada par Variable vs Químico, agregando lineas de tendencia.

La idea con este tipo de gráficas es poder visualizar y entender como es que reacciona cada Variable respecto al incremento/decremento del Químico.

##### **Lineas de Tendencias:**

Se trabajaron dos lineas de tendencia:

1. Lineal: representada por una ecuación de una recta: y=ax + b.
2. Polinomial: representada por una ecuación de un polinomio de grado entre 2 y 4.

#### **Métricas:**

* Dentro de cada gráfica, se podrá visualizar el R^2 de la linea de tendencia correspondiente a una recta. 
* Esta métrica ayuda evualuar que tan buen ajuste tiene la linea de tendencia respecto a todos los puntos.

#### **Outliers:**

Nuevamente se realizo el proceso de filtrado y limpieza de los datos para realizar las gráficas:

1-. **Ceros:** El primer paso es limpiar los datos eliminando todas las filas del par Variable vs Químico, donde la variable contenga un valor igual a 0, ya que representa que no se tomo medida ese día.

2-. **Outliers:** Se identificaron los outliers existentes en cada Variable vs Químico, con un criterio por rango intercuartilico. Se considera un outlier aquel valor que cumpla con una de las sigueintes dos condiciones: 
* El valor es menor que el primer cuartil menos 1.75 veces el rango intercuartilico.
* El valor es mayor que el tercer cuartil más 1.75 veces el rango intercuartilico.

3-. **Caña Recibida:** Se identificaron los casos donde la Caña Recibida no estuvo dentro del rango aceptable o promedio por día. El valor para escoger estos casos fue todo aquel arriba de 23,000. Esto se realizo dado que si no se obtuvo al menos un valor de 23,000 en caña recibida, es muy probable que haya ocurrido imprevistos en producción, como apagones u algun otro inconveniente, convirtiendo esos casos en otro tipo de outliers.

--------------------------

##### **OBSERVACION**: En el titulo de cada gráfica se podrá visualizar si, con la cantidad de datos obtenidos, la relación entre Variable y Químico tiene un comportamiento 'valido' o 'invalido'. 

In [17]:
def graph_model(df,y):
    sep=y.split(sep='_')
    name_y='_'.join(sep[:2])
    
    posx= int(sep[-2])-1
    name_x=df.iloc[:,posx:posx+1].columns[0]
    data_clean=df.drop(df[df[name_x]==0].index)
    q3=data_clean[name_x].quantile(0.75)
    q1=data_clean[name_x].quantile(0.25)
    iqr=q3-q1
    upper=q3+1.75*iqr
    lower=q1-1.75*iqr
    outliers_indx=data_clean[(data_clean[name_x]>upper) | (data_clean[name_x]<lower) | (data_clean['Caña Recibida']<23000)].index
    data_clean=data_clean.drop(outliers_indx)
    
    x_model=data_clean[name_x]
    y_model=data_clean[y]
    
    r_dif=2
    r_last=0
    r_new=0
    values=0
    cont=2
    aux=0
    while (r_dif>0.001) & (cont<4):
        values=0
        cont=cont+1
        r_last=r_new
        fiti=np.polyfit(x_model, y_model, cont)
        for i in range(cont+1):
            values=values+fiti[i]*(x_model**(cont-i))
        corr_matrix=np.corrcoef(y_model,values)
        corr=corr_matrix[0,1]
        r_new=corr**2
        if aux>=1:
            r_dif=r_new-r_last
        else:
            r_dif=2
        aux=aux+1
    poly_result=pd.concat([pd.Series(x_model, name=name_x), pd.Series(values, name=name_y)], axis=1).sort_values(by=name_x)
    
    lineal_model=LinearRegression()
    lineal_model.fit(np.array(x_model).reshape(-1, 1), y_model)
    lineal_predict=lineal_model.coef_[0]*x_model + lineal_model.intercept_
    lineal_result=pd.concat([pd.Series(x_model, name=name_x), pd.Series(lineal_predict, name=name_y)], axis=1).sort_values(by=name_x)
    corr_matrix=np.corrcoef(y_model,lineal_predict)
    corr=corr_matrix[0,1]
    r2=corr**2
    
    if lineal_model.coef_[0]<0:
        if sep[-1]=='-':
            note=' (COMPORTAMIENTO VALIDO)'
        else:
            note=' (COMPORTAMIENTO INVALIDO)'  
    else:
        if sep[-1]=='+':
            note=' (COMPORTAMIENTO VALIDO)'
        else:
            note=' (COMPORTAMIENTO INVALIDO)'    
            
    fig = go.Figure([go.Scatter(x=data_clean[name_x], y=data_clean[y], name='data', mode='markers'),
                        go.Scatter(x=lineal_result[name_x], y=lineal_result[name_y], 
                    name='Linear Trending', mode='lines+markers'),
                        go.Scatter(x=poly_result[name_x], y=poly_result[name_y], 
                    name='Polynomial Grade '+str(cont)+ ' Trending', mode='lines+markers')])
    fig.add_trace(go.Indicator(mode="number",
                                value=r2,
                                title={'text':'R2 Lineal'},
                                domain = {'y': [0.85, 0.9], 'x': [0.92, 0.98]}))
    fig.update_layout(
            title=str(name_y)+' vs '+str(name_x),# + note,
            xaxis_title=name_x,
            yaxis_title=name_y,
            hovermode='x'
            
    )
    fig.show()
        

In [18]:
for i in ys:
    graph_model(df_semana,i)

## Graficando la distribución de puntos para cada par Variable vs Químico, agregando un Modelo de predicción.

Se realizó esta gráfica para poder visualizar si los datos de la muestra recibida para cada par Variable vs Químico son suficientes para poder crear un modelo de predicción adecuado.

##### **Métricas:**

Dado que se utilizaron dos tipos de modelos, similares más no iguales, el algoritmo debía escoger el mejor de ellos, por lo que se decidio tomar como métrica de selección el **RMSE** (Root Mean Square Error), buscando el menor de estos.

#### **Modelos y Proceso:**

Los modelos utilizados fueron:
1. Elastic Net 
2. Linear Regresssión

Para ambos casos el proceso de creación del modelo constaba de 4 pasos:
1. Split de datos: 70% train - 30% test (random)
2. Creación y entrenamiento con el 70% de los datos
3. Test con el 30% de los datos.
4. Realizar predicción de todos los valores del respectivo Químico (x)

#### **T-Test:**

Después de crear los modelos, también se realizo un T-Test, para poder mostrar en una tabla debajo de cada gráfica el valor t y su respectivo valor p, con el cual se puede determinar que parametros del modelos son estadisticamente significativos. Se utilizo un α=0.05 para la evaluación del valor p.

#### **Outliers:**

Nuevamente se realizo el proceso de filtrado y limpieza de los datos para realizar las gráficas:

1-. **Ceros:** El primer paso es limpiar los datos eliminando todas las filas del par Variable vs Químico, donde la variable contenga un valor igual a 0, ya que representa que no se tomo medida ese día.

2-. **Outliers:** Se identificaron los outliers existentes en cada Variable vs Químico, con un criterio por rango intercuartilico. Se considera un outlier aquel valor que cumpla con una de las sigueintes dos condiciones: 
* El valor es menor que el primer cuartil menos 1.75 veces el rango intercuartilico.
* El valor es mayor que el tercer cuartil más 1.75 veces el rango intercuartilico.

3-. **Caña Recibida:** Se identificaron los casos donde la Caña Recibida no estuvo dentro del rango aceptable o promedio por día. El valor para escoger estos casos fue todo aquel arriba de 23,000. Esto se realizo dado que si no se obtuvo al menos un valor de 23,000 en caña recibida, es muy probable que haya ocurrido imprevistos en producción, como apagones u algun otro inconveniente, convirtiendo esos casos en otro tipo de outliers.

--------------------------

##### **OBSERVACION**: En la etiqueta de cada gráfica se podrá visualizar el RMSE obtenido por el modelo escogido. Así también cada modelo fue gráficado con su respectivo intervalo de confianza al 95%.

In [19]:
def graph_training_model(df,y):
    sep=y.split(sep='_')
    name_y='_'.join(sep[:2])
    rmses=[]
    posx= int(sep[-2])-1
    name_x=df.iloc[:,posx:posx+1].columns[0]
    data_clean=df.drop(df[df[name_x]==0].index)
    q3=data_clean[name_x].quantile(0.75)
    q1=data_clean[name_x].quantile(0.25)
    iqr=q3-q1
    upper=q3+1.75*iqr
    lower=q1-1.75*iqr
    outliers_indx=data_clean[(data_clean[name_x]>upper) | (data_clean[name_x]<lower) | (data_clean['Caña Recibida']<23000)].index
    data_clean=data_clean.drop(outliers_indx)
    
    x_model=data_clean[name_x]
    y_model=data_clean[y]

    
    X_train, X_test, y_train, y_test = skmodel.train_test_split(x_model, y_model, train_size= 0.7)
    lineal_model=LinearRegression()
    lineal_model.fit(np.array(X_train).reshape(-1, 1), y_train)
    
    test_predictions=lineal_model.predict(np.array(X_test).reshape(-1, 1))
    rmses.append(math.sqrt(skmetrics.mean_squared_error(y_true=y_test, y_pred=test_predictions)))
    
    total_predict=lineal_model.predict(np.array(x_model).reshape(-1, 1))
    lineal_result=pd.concat([pd.DataFrame({name_x:x_model}).reset_index(drop=True),pd.DataFrame({name_y:total_predict}).reset_index(drop=True)], axis=1).sort_values(by=name_x)
    
    residuals_lineal=y_train - lineal_model.predict(np.array(X_train).reshape(-1, 1))
    ci_lineal= np.quantile(residuals_lineal, 0.95)
    if ci_lineal>=0:
        lineal_result['upper']=lineal_result[name_y]+ci_lineal
        lineal_result['lower']=lineal_result[name_y]-ci_lineal
    else:
        lineal_result['upper']=lineal_result[name_y]-ci_lineal
        lineal_result['lower']=lineal_result[name_y]+ci_lineal
    # ----------------------
    elastic_model=ElasticNet(alpha=0.5, l1_ratio=0.5)
    elastic_model.fit(np.array(X_train).reshape(-1, 1), y_train)
    
    
    test_predictions2=elastic_model.predict(np.array(X_test).reshape(-1, 1))
    rmses.append(math.sqrt(skmetrics.mean_squared_error(y_true=y_test, y_pred=test_predictions2)))
    
    total_predict_2=elastic_model.predict(np.array(x_model).reshape(-1, 1))
    elastic_result=pd.concat([pd.DataFrame({name_x:x_model}).reset_index(drop=True),pd.DataFrame({name_y:total_predict_2}).reset_index(drop=True)], axis=1).sort_values(by=name_x)
    
    residuals_elastic=y_train - elastic_model.predict(np.array(X_train).reshape(-1, 1))
    ci_elastic= np.quantile(residuals_elastic, 0.95)
    if ci_elastic>=0:
        elastic_result['upper']=elastic_result[name_y]+ci_elastic
        elastic_result['lower']=elastic_result[name_y]-ci_elastic
    else:
        elastic_result['upper']=elastic_result[name_y]-ci_elastic
        elastic_result['lower']=elastic_result[name_y]+ci_elastic
    #------------------------
    for_ttest=sm.add_constant(X_train)
    model_ttest = sm.OLS(y_train, for_ttest).fit()
    result_ttest=pd.DataFrame({'Parameters':model_ttest.tvalues.index, 't':model_ttest.tvalues.values, 'p_value':model_ttest.pvalues.values})
    result_ttest['Conclusion']=np.where(result_ttest['p_value']<0.05,
                                        'p_value<=α, se rechaza la Hipotesis nula, el parametro es estadisticamente significativo para el modelo',
                                        'p_value>α, no se rechaza la Hipotesis nula, el parametro no es estadisticamente significativo para el modelo')    
    
    
    fig = go.Figure(go.Scatter(x=data_clean[name_x], y=data_clean[y], name='data', mode='markers'))
    if rmses.index(min(rmses))==0:
            fig.add_trace(go.Scatter(x=lineal_result[name_x], y=lineal_result[name_y], name='Lineal Model, RMSE: '+str(min(rmses)), mode='lines+markers'))
            fig.add_trace(go.Scatter(name='Lower Bound',
                                x=lineal_result[name_x],
                                y=lineal_result['lower'],
                                marker=dict(color="#444"),
                                line=dict(width=0),
                                mode='lines',
                                fillcolor='rgba(68, 68, 68, 0.3)',
                                fill='tonexty',
                                showlegend=False))
            fig.add_trace(go.Scatter(name='Upper Bound',
                                x=lineal_result[name_x],
                                y=lineal_result['upper'],
                                marker=dict(color="#444"),
                                line=dict(width=0),
                                mode='lines',
                                fillcolor='rgba(68, 68, 68, 0.3)',
                                fill='tonexty',
                                showlegend=False))
    elif rmses.index(min(rmses))==1:
            fig.add_trace(go.Scatter(x=elastic_result[name_x], y=elastic_result[name_y], name='Elastic Net Model, RMSE: '+str(min(rmses)), mode='lines+markers'))
            fig.add_trace(go.Scatter(name='Lower Bound',
                                x=elastic_result[name_x],
                                y=elastic_result['lower'],
                                marker=dict(color="#444"),
                                line=dict(width=0),
                                mode='lines',
                                fillcolor='rgba(68, 68, 68, 0.3)',
                                fill='tonexty',
                                showlegend=False))
            fig.add_trace(go.Scatter(name='Upper Bound',
                                x=elastic_result[name_x],
                                y=elastic_result['upper'],
                                marker=dict(color="#444"),
                                line=dict(width=0),
                                mode='lines',
                                fillcolor='rgba(68, 68, 68, 0.3)',
                                fill='tonexty',
                                showlegend=False))
    fig.update_layout(
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            ),
            title=str(name_y)+' vs '+str(name_x),
            xaxis_title=name_x,
            yaxis_title=name_y,
            hovermode='x'
    )
    table_t = go.Figure(data=[go.Table(
        header=dict(values=list(['Parameters', 'T', 'p_value', 'Conclusion']),
                    align='center'),
        cells=dict(values=[result_ttest['Parameters'],result_ttest['t'], result_ttest['p_value'], result_ttest['Conclusion']],
                align='center')),
        
    ])
    table_t.update_layout(
            title='T-Test for Model of '+str(name_y)+' vs '+str(name_x)
    )
    sub_fig3 = make_subplots(
                rows=2, cols=1,
                shared_xaxes=True,
                vertical_spacing=0.25,
                specs=[[{"type": "scatter"}],
                    [{"type": "table"}]]
            )

    sub_fig3 = sub_fig3.add_trace(fig.data[0], row=1, col=1)
    sub_fig3 = sub_fig3.add_trace(fig.data[1], row=1, col=1)
    sub_fig3 = sub_fig3.add_trace(fig.data[2], row=1, col=1)
    sub_fig3 = sub_fig3.add_trace(fig.data[3], row=1, col=1)
    sub_fig3 = sub_fig3.add_trace(table_t.data[0], row=2, col=1)
    sub_fig3.update_layout(
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            ),
            height=620,
            title=str(name_y)+' vs '+str(name_x),
            xaxis_title=name_x,
            yaxis_title=name_y,
            hovermode='x')
    sub_fig3.show()
        

In [20]:
for i in ys:
    graph_training_model(df_semana,i)